In [122]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 

In [197]:
import rankingmethod
import rankingmeasure
import imp
imp.reload(rankingmethod)
imp.reload(rankingmeasure)

<module 'rankingmeasure' from '/home/george/Documents/diploma/code/rankingmeasure.py'>

In [49]:
def all_measures(train, test, method, skip_train=False, top=None):
    print("precision5", rankingmeasure.get_prec_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("1-call5", rankingmeasure.get_prec_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("MRR", rankingmeasure.get_MRR(train, test, 
            method, skip_train=skip_train, top=top))

In [39]:
def make_valid_data(data, thres=25, reindex=True):
    list_good_user = list(np.where(np.bincount(data[:, 0]) > thres)[0])
    set_good_user = set(list_good_user)
    epinion = []
    for i,j in data:
        if i in set_good_user:
            epinion.append([i, j])
    epinion = np.array(epinion)
    if reindex:
        for new_u, u in enumerate(np.sort(list_good_user)):
            epinion[epinion[:, 0] == u, 0] = new_u
        print(epinion.shape)
        list_good_item = np.unique(epinion[:, 1])
        for new_i, i in enumerate(np.sort(list_good_item)):
            epinion[epinion[:, 1] == i, 1] = new_i
    user_item = np.max(epinion, axis=0) + 1
    return epinion, user_item


def create_listarray(data):
    xlist = []
    for u in np.sort(np.unique(data[:, 0])):
        xlist.append(data[data[:, 0] == u, 1])
    return xlist

In [149]:
def givenK_train_test(data, K):
    xlist = create_listarray(data)
    
    train = []
    test = []
    for x in xlist:
        train_ind = np.random.choice(x.shape[0], K, replace=False)
        tmp = np.ones(x.shape[0])
        tmp[train_ind] = 0
        test_ind = np.arange(x.shape[0])[tmp == 1]
        
        train.append(x[train_ind])
        test.append(x[test_ind])
        
    return (train, test, xlist)

##Мои данные

In [196]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [303]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25
#Также проводится переиндексанция пользователей и предметов

In [ ]:
epinion = make_valid_data(data)

In [ ]:
train, test, traintest = givenK_train_test(epinion, 5)

In [ ]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

In [345]:
list_reg = [0.01, 0.1 ,0.5]
list_lrate = [0.001, 0.005, 0.01, 0.05, 0.1 ]
list_K = [10, 5, 7, 12, 15]

for K in list_K:
    for lrate in list_lrate:
        for reg in list_reg:
            climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=100)
            climf.fit(train)
            print(K, lrate, reg)
            print(rankingmeasure.get_MRR(train, test, 
            climf, skip_train=False, top=top))

10 0.001 0.01
0.0068061377633100405
10 0.001 0.1
0.008031961359121426
10 0.001 0.5
0.005912269091311202
10 0.005 0.01
0.007815140434096543
10 0.005 0.1
0.00652022706541012
10 0.005 0.5
0.006532552664074765
10 0.01 0.01
0.007028481810001253
10 0.01 0.1
0.005199687139427464
10 0.01 0.5
0.00642130218487009
10 0.05 0.01
0.007314500601830544
10 0.05 0.1
0.00563555140338767
10 0.05 0.5
0.005847528094546522
10 0.1 0.01
0.008851060117673922
10 0.1 0.1
0.008597647640724496
10 0.1 0.5
0.007773293622440457
5 0.001 0.01
0.008755247464899283
5 0.001 0.1
0.010599960371667312
5 0.001 0.5
0.006476902717603026
5 0.005 0.01
0.010910411353978392
5 0.005 0.1
0.005833571238977291
5 0.005 0.5
0.008890794054437025
5 0.01 0.01
0.015242684443990519
5 0.01 0.1
0.006276795759708992
5 0.01 0.5
0.005705023945597651
5 0.05 0.01
0.022483016113061575
5 0.05 0.1
0.005442549602997411
5 0.05 0.5
0.008115720609231233
5 0.1 0.01
0.019875807579590635
5 0.1 0.1
0.009115952585898184
5 0.1 0.5
0.01036981495442007
7 0.001 0.01

KeyboardInterrupt: 

##Данные создателей CLiMF

In [54]:
tmp = mmread("../datasets/epinions_original/EP25_UPL5_test.mtx")
test = np.array(list(zip(tmp.row, tmp.col)))
tmp = mmread("../datasets/epinions_original/EP25_UPL5_train.mtx")
train = np.array(list(zip(tmp.row, tmp.col)))
user_item = [max(np.max(test[:,0]), np.max(train[:, 0])) + 1, max(np.max(test[:,1]), np.max(train[:,1])) + 1]

In [55]:
train = create_listarray(train)
test  = create_listarray(test)

In [77]:
randomrec = rankingmethod.RandomRec(user_item)
randomrec.fit(train)
all_measures(train, test, randomrec, False)

precision5 0.00148367952522
1-call5 0.00135650699449
MRR 0.009444793407610922


In [51]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

precision5 0.0538787621874
1-call5 0.0538787621874
MRR 0.11273876816174451
precision5 0.171131835524
1-call5 0.171131835524
MRR 0.4020232337666698


In [ ]:
climf = rankingmethod.CLiMF(user_item, maxiter=40, lrate = 0.1, verbose=2)
climf.fit(train)


In [106]:
all_measures(train, test, climf, False)

precision5 0.0635438745231
1-call5 0.0635438745231
MRR 0.16061628627384766


In [113]:
all_measures(train, test, climf, True, top)

precision5 0.0412038999576
1-call5 0.0412038999576
MRR 0.11461895836301948


In [123]:
list_reg = [0.001, 0.002, 0.005, 0.0005]
list_lrate = [0.05, 0.07, 0.1, 0.12, 0.15 ]
list_K = [10, 12, 15, 7, 5]
times_count = 3

for K, lrate, reg in itertools.product(list_K, list_lrate, list_reg):
    MRR = 0
    
    for t in range(times_count):
        climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=50)
        climf.fit(train)
        MRR += rankingmeasure.get_MRR(train, test, climf, skip_train=False)
    print("K", K, "lrate", lrate, "reg", reg, "MRR", MRR/times_count)
    sys.stdout.flush()

K 10 lrate 0.05 reg 0.001 MRR 0.16740641961680544
K 10 lrate 0.05 reg 0.002 MRR 0.1632928986028329
K 10 lrate 0.05 reg 0.005 MRR 0.1527730041694714
K 10 lrate 0.05 reg 0.0005 MRR 0.1622970666170243
K 10 lrate 0.07 reg 0.001 MRR 0.1617021688446262
K 10 lrate 0.07 reg 0.002 MRR 0.15214149672638577
K 10 lrate 0.07 reg 0.005 MRR 0.1428759872783604
K 10 lrate 0.07 reg 0.0005 MRR 0.1558988328476956
K 10 lrate 0.1 reg 0.001 MRR 0.1609389294595767
K 10 lrate 0.1 reg 0.002 MRR 0.14313962174434888
K 10 lrate 0.1 reg 0.005 MRR 0.12702524191410772
K 10 lrate 0.1 reg 0.0005 MRR 0.1433087707886965
K 10 lrate 0.12 reg 0.001 MRR 0.14271491223696034
K 10 lrate 0.12 reg 0.002 MRR 0.13561491669555753
K 10 lrate 0.12 reg 0.005 MRR 0.12988356090059913
K 10 lrate 0.12 reg 0.0005 MRR 0.13972362078679446
K 10 lrate 0.15 reg 0.001 MRR 0.06756475308278055
K 10 lrate 0.15 reg 0.002 MRR 0.06772883536587512
K 10 lrate 0.15 reg 0.005 MRR 0.06770193334254045
K 10 lrate 0.15 reg 0.0005 MRR 0.06781537216288246
K 12 lr

In [131]:
np.random.choice([0,2,5,1], 5)

array([1, 1, 1, 5, 1])

In [280]:
bpr_mf = rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=10, verbose=3 )

In [279]:
import rankingmethod
import rankingmeasure
import imp
imp.reload(rankingmethod)
imp.reload(rankingmeasure)

<module 'rankingmeasure' from '/home/george/Documents/diploma/code/rankingmeasure.py'>

In [278]:
random.choice(np.array([1, 2, 4, 5234, 1]))

5234

In [258]:
bpr_mf.fit(train)

23590
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'int'>
[[ 0.01394466  0.05252042  0.06346805  0.05412942  0.06440757  0.05854368
   0.01151907  0.00469982  0.03394135  0.09979507  0.02939081  0.00785782
   0.09233927  0.00722193  0.00832375  0.00900277  0.04647272  0.05679726
   0.09599775  0.042029  ]] <class 'numpy.ndarray'>
[[ 0.01041565  0.08999106  0.05318125  0.03954916  0.0563052   0.00531645
   0.03422625  0.05317337  0.03014774  0.08123811  0.07037154  0.00065046
   0.06983113  0.01572561  0.07067846  0.02439877  0.0472366   0.03246325
   0.04884406  0.00870717]] <class 'numpy.ndarray'>
[ 0.09291668  0.02429352  0.02731799  0.05265685  0.01890156  0.01282934
  0.03175827  0.07517802  0.0459524   0.06716854  0.0778874   0.09125913
  0.09872893  0.07120572  0.04984937  0.02247315  0.07500873  0.07636591
  0.00896483  0.0238435 ] <class 'numpy.ndarray'>
(1, 20) (1, 20) (20,)
wtf?
